## Getting corpus using web scrapping
- HTML Parsing

## Naive Bayes Classification

Tarea: Obtener las canciones de

- https://genius.com/albums/Olivia-rodrigo/Sour
- https://genius.com/albums/Taylor-swift/Fearless


Crear un clasificador de Naive Bayes, para ver si se parecen las canciones de Olivia Rodrigo

Notes:
- With Bayes, we assume independance for each event

### Web scrapping 

In [6]:
!pip install requests spacy

In [17]:
import requests
import spacy
from bs4 import BeautifulSoup as soup

In [18]:
headers = {
    'User-Agent':'Mozilla/5.0'
    
}

sour_url = 'https://genius.com/albums/Olivia-rodrigo/Sour'
fearless_url = 'https://genius.com/albums/Taylor-swift/Fearless'

In [19]:
def get_soup(url,headers):
    """Get HTML parser"""
    response = requests.get(url,headers=headers)
    web_page = response.text
    return soup(web_page,"html.parser")

In [20]:
def get_tracks(url,headers):
    page_soup = get_soup(url=url,headers=headers)
    return page_soup.find_all("div","chart_row-content") # all divs that contain chart...


In [21]:
def get_lyrics(url,headers):
    lyr= None
    while lyr ==None:
        page_soup = get_soup(url=url,headers=headers)
        lyr = page_soup.find("div","song_body-lyrics")
    return lyr.text.replace('\u200b','').strip() # eliminate spaces

In [24]:
def list_lyrics(url,headers):
    tracks = get_tracks(url,headers)
    links = [{
     "title":title.find("h3").next.strip().replace('\u200b',''),
    "link":title.find("a")["href"]
    } for title in tracks]
    return [{
        "title":t["title"],
        "lyrics":get_lyrics(url =t["link"],headers=headers)
    } for t in links]

In [25]:
sour_lyrics = list_lyrics(sour_url,headers)

In [26]:
fearless_lyrics = list_lyrics(fearless_url,headers)

In [30]:
print(len(fearless_lyrics))

14


In [32]:
print(fearless_lyrics[2]["lyrics"])

Love Story Lyrics



[Verse 1]
We were both young when I first saw you
I close my eyes, and the flashback starts
I'm standing there
On a balcony in summer air
See the lights, see the party, the ball gowns
See you make your way through the crowd
And say, "Hello"
Little did I know

[Pre-Chorus]
That you were Romeo, you were throwing pebbles
And my daddy said, "Stay away from Juliet"
And I was crying on the staircase
Begging you, "Please don't go," and I said

[Chorus]
Romeo, take me somewhere we can be alone
I'll be waiting, all there's left to do is run
You'll be the prince and I'll be the princess
It's a love story, baby, just say "Yes"

[Verse 2]
So I sneak out to the garden to see you
We keep quiet, 'cause we're dead if they knew
So close your eyes
Escape this town for a little while, oh, oh

[Pre-Chorus]
'Cause you were Romeo, I was a scarlet letter
And my daddy said, "Stay away from Juliet"
But you were everything to me
I was begging you, "Please don't go," and I said

[Chorus]
Rom

In [33]:
# cleaning texts
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
print(len(sour_lyrics))
print(len(fearless_lyrics))

11
14


In [37]:
print(sour_lyrics[0]["lyrics"])

brutal Lyrics



[Intro]
I want it to be, like, messy

[Verse 1]
I'm so insecure, I think
That I'll die before I drink
And I'm so caught up in the news
Of who likes me and who hates you
And I'm so tired that I might
Quit my job, start a new life
And they'd all be so disappointed
'Cause who am I if not exploited?
And I'm so sick of seventeen
Where's my fucking teenage dream?
If someone tells me one more time
"Enjoy your youth," I'm gonna cry
And I don't stick up for myself
I'm anxious, and nothing can help
And I wish I'd done this before
And I wish people liked me more

[Chorus]
All I did was try my best
This the kinda thanks I get?
Unrelentlessly upset (Ah-ah-ah)
They say these are the golden years
But I wish I could disappear
Ego crush is so severe
God, it's brutal out here

[Post-Chorus]
(Yeah)

[Verse 2]
I feel like no one wants me
And I hate the way I'm perceived
I only have two real friends
And lately, I'm a nervous wreck
'Cause I love people I don't like
And I hate every song I w

In [40]:
# Create dataset
#sour_lyrics
fearless_lyrics[3]

{'title': 'Hey Stephen',
 'lyrics': "Hey Stephen Lyrics\n\n\n\n[Intro]\nMmm-mm, mm-mm\nMmm-mm, mm-mm\nMmm-mm, mm-mm, yeah\n\n[Verse 1]\nHey Stephen, I know looks can be deceiving\nBut I know I saw a light in you\nAnd as we walked we were talking\nI didn't say half the things I wanted to\nOf all the girls tossing rocks at your window\nI'll be the one waiting there even when it's cold\nHey Stephen, boy, you might have me believing\nI don't always have to be alone\n\n[Chorus]\n'Cause I can't help it if you look like an angel\nCan't help it if I wanna kiss you in the rain so\nCome feel this magic I've been feeling since I met you\nCan't help it if there's no one else\nMmm, I can't help myself\n\n[Verse 2]\nHey Stephen, I've been holding back this feeling\nSo I got some things to say to you\nI've seen it all, so I thought\nBut I never seen nobody shine the way you do\nThe way you walk, way you talk, way you say my name\nIt's beautiful, wonderful, don't you ever change\nHey Stephen, why are 

In [46]:
dataset = []

for sour_lyric in sour_lyrics:
    dataset.append([sour_lyric["lyrics"],"Sour"])
    
for fear_lyric in fearless_lyrics:
    dataset.append([fear_lyric["lyrics"],"Fearless"])

In [47]:
dataset[-1]

['Fearless [Liner Notes] Lyrics\n\n\n\nTaylor\'s Letter\nThis album is called "Fearless", and I guess I\'d like to clarify why we chose that as the title. To me, "Fearless" is not the absence of fear. It\'s not being completely unafraid. To me, fearless is having fears. Fearless is having doubts. Lots of them. To me, fearless is living in spite of those things that scare you to death. Fearless is falling madly in love again, even though you\'ve been hurt before. Fearless is walking into your freshman year of high school at fifteen. Fearless is getting back up and fighting for what you want over and over again...even though every time you\'e tried before, you\'ve lost. It\'s fearless to have faith that someday things will change. Fearless is having the courage to say goodbye to someone who only hurts you, even if you can\'t breathe without them. I think it\'s fearless to fall for your best friend, even though he\'s in love with someone else. And when someone apologizes to you enough tim

In [51]:
dataset_pd = pd.DataFrame(dataset)
dataset_pd.columns = ["Lyrics", "Album"]

In [52]:
dataset_pd

,Lyrics,Album
0,"brutal Lyrics\n\n\n\n[Intro]\nI want it to be,...",Sour
1,traitor Lyrics\n\n\n\n[Intro]\nOoh-ooh-ooh\nAh...,Sour
2,drivers license Lyrics\n\n\n\n[Verse 1]\nI got...,Sour
3,"1 step forward, 3 steps back Lyrics\n\n\n\n[Ve...",Sour
4,deja vu Lyrics\n\n\n\n[Verse 1]\nCar rides to ...,Sour
5,good 4 u Lyrics\n\n\n\n[Intro]\n(Ah)\n\n[Verse...,Sour
6,enough for you Lyrics\n\n\n\n[Verse 1]\nI wore...,Sour
7,happier Lyrics\n\n\n\n[Verse 1]\nWe broke up a...,Sour
8,"jealousy, jealousy Lyrics\n\n\n\n[Verse 1]\nI ...",Sour
9,favorite crime Lyrics\n\n\n\n[Verse 1]\nKnow t...,Sour


In [63]:
nltk.download('stopwords')
corpus = []
  
for i in range(0, 25):
    text = re.sub('[^a-zA-Z]', ',', dataset_pd['Lyrics'][i])
    text = text.lower()
    text = text.split()
    ps = PorterStemmer()
    text = ''.join(text)
    corpus.append(text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fjdur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
corpus

['brutal,lyrics,,,,,intro,,i,want,it,to,be,,like,,messy,,,verse,,,,i,m,so,insecure,,i,think,that,i,ll,die,before,i,drink,and,i,m,so,caught,up,in,the,news,of,who,likes,me,and,who,hates,you,and,i,m,so,tired,that,i,might,quit,my,job,,start,a,new,life,and,they,d,all,be,so,disappointed,,cause,who,am,i,if,not,exploited,,and,i,m,so,sick,of,seventeen,where,s,my,fucking,teenage,dream,,if,someone,tells,me,one,more,time,,enjoy,your,youth,,,i,m,gonna,cry,and,i,don,t,stick,up,for,myself,i,m,anxious,,and,nothing,can,help,and,i,wish,i,d,done,this,before,and,i,wish,people,liked,me,more,,,chorus,,all,i,did,was,try,my,best,this,the,kinda,thanks,i,get,,unrelentlessly,upset,,ah,ah,ah,,they,say,these,are,the,golden,years,but,i,wish,i,could,disappear,ego,crush,is,so,severe,god,,it,s,brutal,out,here,,,post,chorus,,,yeah,,,,verse,,,,i,feel,like,no,one,wants,me,and,i,hate,the,way,i,m,perceived,i,only,have,two,real,friends,and,lately,,i,m,a,nervous,wreck,,cause,i,love,people,i,don,t,like,and,i,hate,every,song,i

In [66]:
# creating bag of words model
cv = CountVectorizer(max_features = 1500)
  
X = cv.fit_transform(corpus).toarray()
y = dataset_pd.iloc[:, 1].values

In [85]:
my_test= "i,know,we were,perfect,and,i,can,imagine,that,you,write,my,song,car,paco"
o = cv.fit_transform([my_test]).toarray()

In [86]:
o

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)

In [72]:
print(len(X))
print(len(X[0]))

print(X[100])
print(y[0])

25
1152
[[0 0 0 ... 1 0 1]
 [0 1 0 ... 3 0 0]
 [0 6 0 ... 7 0 0]
 ...
 [0 0 0 ... 3 0 0]
 [0 0 0 ... 2 0 0]
 [0 0 1 ... 3 1 0]]
Sour


In [74]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=3db8f9d0f2bab5a7cad5005758704a1b80f00a0417470dc0b3d2fa4223754de5
  Stored in directory: c:\users\fjdur\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
           X, y, test_size = 0.25, random_state = 1)

In [80]:
print(y_train)
print(y_test)

['Fearless' 'Sour' 'Sour' 'Fearless' 'Sour' 'Sour' 'Fearless' 'Sour'
 'Fearless' 'Sour' 'Fearless' 'Fearless' 'Fearless' 'Sour' 'Sour'
 'Fearless' 'Fearless' 'Sour']
['Fearless' 'Fearless' 'Fearless' 'Sour' 'Fearless' 'Sour' 'Fearless']


In [81]:


# fitting naive bayes to the training set
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
  
classifier = GaussianNB();
classifier.fit(X_train, y_train)
  
# predicting test set results
y_pred = classifier.predict(X_test)
  
# making the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[5, 0],
       [0, 2]], dtype=int64)

In [ ]:

  
dataset = [["I liked the movie", "positive"],
           ["It’s a good movie. Nice story", "positive"],
           ["Hero’s acting is bad but heroine looks good.\
            Overall nice movie", "positive"],
            ["Nice songs. But sadly boring ending.", "negative"],
            ["sad movie, boring movie", "negative"]]
              
dataset = pd.DataFrame(dataset)
dataset.columns = ["Text", "Reviews"]
  
nltk.download('stopwords')
  
corpus = []
  
for i in range(0, 5):
    text = re.sub('[^a-zA-Z]', '', dataset['Text'][i])
    text = text.lower()
    text = text.split()
    ps = PorterStemmer()
    text = ''.join(text)
    corpus.append(text)
  
# creating bag of words model
cv = CountVectorizer(max_features = 1500)
  
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

# splitting the data set into training set and test set
from sklearn.cross_validation import train_test_split
  
X_train, X_test, y_train, y_test = train_test_split(
           X, y, test_size = 0.25, random_state = 0)

# fitting naive bayes to the training set
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
  
classifier = GaussianNB();
classifier.fit(X_train, y_train)
  
# predicting test set results
y_pred = classifier.predict(X_test)
  
# making the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [4]:
ham = 'play sports today, went play sports, secret sports event, sports is today, sports costs money'
spam = 'offer is secret, click secret link, secret sports link'

In [3]:
V = ['offer', 'is', 'secret', 'click', 'link', 'sports', 'play', 'today', 'went', 'event', 'costs', 'money']

In [ ]:
#ne
V_dict= {'offer':2, 'is':3, 'secret':7, 'click':2, 'link':4, 'sports', 'play', 'today', 'went', 'event', 'costs', 'money'}

In [ ]:
# Modelo bolsa de palabra,
V = {offer: 2, is: 3, secret: 7, click: 2, link: 4, sports: 7, play: 2, today: 2, went: 1, event: 1, costs: 1, money: 1}

V = {offer: 1, is: 2, secret: 4, click: 1, link: 2, sports: 6, play: 2, today: 2, went: 1, event: 1, costs: 1, money: 1}